# Tic-tac-toe minmax algorithm with alpha beta pruning

In [ ]:
import numpy as np

The original code has been extended with the implementation of the minmax algorithm and with alpha beta pruning.

The **minmax** method has been changed to now have alpha, beta

In [ ]:
class State:
    """ Capturing the state of the game
    gameplan - two-dimensional 3x3 array
             - 0 - empty array
             - 1 - X
             - 2 - O
    player - the player who has the turn in the game

    current_player - player who is on the turn in the given state when searching the state space
    depth - depth of the state space search
    max_depth - maximum length of the search
    """

    generated = 0

    def __init__(self, gameplan, player, current_player=None, depth=0, max_depth=100):
        self.gameplan = gameplan
        self.player = player
        if current_player is None:
            self.current_player = player
        else:
            self.current_player = current_player
        self.depth = depth
        self.max_depth = max_depth
        
        State.generated += 1

    def terminal_test(self):
        """ The method tests the current state and returns a value indicating whether the game is finished and, if so, who the winner is

            0 - no final status
            1 - 1 wins
            2 - winner 2
        """

        # end of game rules - horizontal and vertical triplets
        for i in range(3):
            if np.array_equal(self.gameplan[i], [1, 1, 1]):
                return 1
            if np.array_equal(self.gameplan[i], [2, 2, 2]):
                return 2
            if np.array_equal(self.gameplan[:, i], [1, 1, 1]):
                return 1
            if np.array_equal(self.gameplan[:, i], [2, 2, 2]):
                return 2

        # end of game rules - diagonals
        if np.array_equal(self.gameplan.diagonal(), [1, 1, 1]):
            return 1
        if np.array_equal(self.gameplan.diagonal(), [2, 2, 2]):
            return 2
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [1, 1, 1]):
            return 1
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [2, 2, 2]):
            return 2

        return 0

    def utility(self, result):
        """ The method tests the current state and returns a value indicating whether the game is finished and, if so, who the winner is

            0 - no final status
            1 - 1 wins
            2 - winner 2
        """
        if result == 0:
            return 0
        elif result == self.player:
            return 1
        else:
            return -1

    def possible_actions(self):
        """ The method returns a list of possible actions for the given state
            The action is expressed by the coordinates of an empty playing field.
        """
        possible_actions = []
        # Finding empty playing fields
        for i in range(3):
            for j in range(3):
                if self.gameplan[i][j] == 0:
                    possible_actions.append((i, j))
        return possible_actions

    def expand(self, select_action):
        """ The method creates a new game state by applying the selected action to the current state

            In the new state, the opposing player will have the turn, but the state will be evaluated from the original player's perspective
            The depth of the searched state space will also increase
        """
        # coordinate check
        if select_action[0] not in range(3):
            return None
        if select_action[1] not in range(3):
            return None

        # playing field must be empty
        if self.gameplan[select_action[0], select_action[1]] != 0:
            return None
        
        new_array = np.copy(self.gameplan)
        new_array[select_action[0], select_action[1]] = self.current_player
        return State(new_array, 
                     self.player, 
                     self.next_current_player(), 
                     self.depth + 1, 
                     max_depth=self.max_depth)

    def minmax(self, strategy="max", alfa=float('-inf'), beta= float('inf')):
        """"
        The method selects the one that matches the strategy from the possible actions for the given game state

        stategy - what strategy will be used to select from the possible actions
        alpha - algorithm parameter
        beta - algorithm parameter
        """
        
        # checking the state of the game, for a completed game the rating from the utility method is returned
        result = self.terminal_test()
        if result != 0:            
            return self.utility(result), action       

        # initialization of values for each strategy
        if strategy == "max":
            selected_utilization_value = float('-inf')
            next_strategy = "min"
        else:
            selected_utilization_value = float('inf')
            next_strategy = "max"

        # finding possible actions
        actions = self.possible_actions()

        # the selected action is filled with the first action of the possible actions
        selected_action = actions[0]

        
        # finding the optimum action from all possible acttions
        for action in actions:

            # state expansion
            expanded_state = self.expand(action)

            # game end check for expanded condition
            result = expanded_state.terminal_test()

            if result != 0:
                # games end, return status and action rating
                return expanded_state.utility(result), action
            else:
                # the game continues
                # if there is at least one state in the expanded state, continue to expand the state, otherwise it's a draw
                if len(expanded_state.possible_actions()) == 0:
                    utilization = 0
                else:
                    # recursive call
                    # !!! todo - add search depth limitation
                    utilization, _ = expanded_state.minmax(next_strategy, alfa, beta)

                # according to the strategy, the evaluated action is chosen as the selected action
                if strategy == "max":
                
                    if utilization > selected_utilization_value: 
                        selected_utilization_value = utilization
                        selected_action = action
                        
                    # if the returned minmax value is greater than beta, return it
                    if selected_utilization_value >= beta:
                        return selected_utilization_value, selected_action
                    
                    # if the returned value is greater than alpha, update alpha
                    if selected_utilization_value > alfa:
                        alfa = selected_utilization_value
                else:
                    # strategy min
                    if utilization < selected_utilization_value:
                        selected_utilization_value = utilization
                        selected_action = action
                        
                    # if the returned minmax value is less than alpha, return it
                    if selected_utilization_value <= alfa:
                        return selected_utilization_value, selected_action
                    
                    # if the returned value is less than beta, update the beta
                    if selected_utilization_value < beta:
                        beta = selected_utilization_value
                        
        return selected_utilization_value, selected_action

    def next_current_player(self):
        """ The method returns the opponent's for state space searching
        """
        return 3 - self.current_player

    def next_player(self):
        """ Method returns opponent
        """
        return 3 - self.player

# Game of tic-tac-toe

Creating the initial state of the game
* Game plan is empty (0)
* Game 1 is on the turn

In [ ]:
state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
              player=1)

The cycle of the game remained unchanged. 

Run the turn and compare the result of the turn, times and numbers of generated states against the original minmax version.

In [ ]:
while True:
    # Check that the game is not over
    game_result = state.terminal_test()
    if game_result != 0:
        print(f"Winner is {game_result} ")
        break

    # Checking for drawn
    if len(state.possible_actions()) == 0:
        print("Drawn")
        break

    # player's move
    print(f"=====================\nPlayer {state.player}")
    _, player_action = state.minmax("max")
    print(f"Select action: {player_action}")
    state = state.expand(player_action)
    print(state.gameplan)
    print(f"Generated states {State.generated}.")
    State.generated = 0

    # switching the game to the other player
    state.player = state.next_player()

# Task
Add a constraint to the algorithm to limit the maximum search depth.

Again, you need to change the code in place of # **!!! todo**

In [ ]:
state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
              player=1, max_depth=2)

In [ ]:
while True:
    # Check that the game is not over
    game_result = state.terminal_test()
    if game_result != 0:
        print(f"Winner is {game_result} ")
        break

    # Checking for drawn
    if len(state.possible_actions()) == 0:
        print("Drawn")
        break

    # player's move
    print(f"=====================\nPlayer {state.player}")
    _, player_action = state.minmax("max")
    print(f"Select action: {player_action}")
    state = state.expand(player_action)
    print(state.gameplan)
    print(f"Generated states {State.generated}.")
    State.generated = 0

    # switching the game to the other player
    state.player = state.next_player()